In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (17901 Collins Ave #1104, Sunny Isles Beach, F...
2     (1000 Biscayne Blvd #3101, Miami, FL 33132, US...
3     (16901 Collins Ave #1403, Sunny Isles Beach, F...
4     (17875 Collins Ave #3504, Sunny Isles Beach, F...
5     (10201 E Bay Harbor Dr Apt 607, Bay Harbor Isl...
6     (17111 Biscayne Blvd Unit 2110, Aventura, FL 3...
7     (16001 Collins Ave APT 2704, Sunny Isles Beach...
8     (15811 Collins Ave APT 1606, Sunny Isles Beach...
9     (2900 NE 7th Ave Unit 3702, Miami, FL 33137, U...
10    (201 Crandon Blvd APT 1028, Key Biscayne, FL 3...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Estates at Acqualina\r17901 Collins Ave 1104\r...,11/21/2022,51,"$7,823,000.00",4130,"$1,894.19",Michael Goldstein,Acqualina Realty,Gil Brendel,"G Realty Group, LLC.",Estates at Acqualina 17901 Collins Ave 1104 Su...,Estates at Acqualina,17901 Collins Ave 1104 Sunny Isles Beach,"(17901 Collins Ave #1104, Sunny Isles Beach, F...","(25.9419666, -80.1210341, 0.0)",25.941967,-80.121034,0.0
2,One Thousand Museum\r1000 Biscayne Blvd 3101\r...,11/22/2022,460,"$7,200,000.00",4600,"$1,565.22",Lana Bell,Douglas Elliman,Michael Duchon,The Corcoran Group,One Thousand Museum 1000 Biscayne Blvd 3101 Miami,One Thousand Museum,1000 Biscayne Blvd 3101 Miami,"(1000 Biscayne Blvd #3101, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
3,Jade Signature\r16901 Collins Ave 1403\rSunny ...,11/21/2022,138,"$3,800,000.00",2501,"$1,519.39",Anastasia Cucu,Douglas Elliman,Jose Santovena,Luxe Living Realty,Jade Signature 16901 Collins Ave 1403 Sunny Is...,Jade Signature,16901 Collins Ave 1403 Sunny Isles Beach,"(16901 Collins Ave #1403, Sunny Isles Beach, F...","(25.9328476, -80.1211362, 0.0)",25.932848,-80.121136,0.0
4,Acqualina Ocean Residence\r17875 Collins Ave 3...,11/21/2022,224,"$2,900,000.00",2031,"$1,427.87",Denise Rubin,Coldwell Banker Realty,NON-MLS MEMBER,MAR NON MLS MEMBER,Acqualina Ocean Residence 17875 Collins Ave 35...,Acqualina Ocean Residence,17875 Collins Ave 3504 Sunny Isles Beach,"(17875 Collins Ave #3504, Sunny Isles Beach, F...","(25.940858, -80.1208766, 0.0)",25.940858,-80.120877,0.0
5,Sereno Residences Condo\r10201 E Bay Harbor Dr...,11/21/2022,59,"$2,800,000.00",2105,"$1,330.17",Ana Paula Lemes Alves,Miami New Realty,Armando Romero,BrokerNation Real Estate,Sereno Residences Condo 10201 E Bay Harbor Dr ...,Sereno Residences Condo,10201 E Bay Harbor Dr 607 Bay Harbor Islands,"(10201 E Bay Harbor Dr Apt 607, Bay Harbor Isl...","(25.8926397, -80.13107749999999, 0.0)",25.892640,-80.131077,0.0
6,Marina Palms Residences\r17111 Biscayne Blvd 2...,11/21/2022,161,"$2,175,000.00",2386,$911.57,Jose Augusto Nunes,Algebra Realty.com,Erin Argiropoulos,"Compass Florida, LLC",Marina Palms Residences 17111 Biscayne Blvd 21...,Marina Palms Residences,17111 Biscayne Blvd 2110 North Miami Beach,"(17111 Biscayne Blvd Unit 2110, Aventura, FL 3...","(25.9342057, -80.1486663, 0.0)",25.934206,-80.148666,0.0
7,Trump Tower One\r16001 Collins Ave 2704\rSunny...,11/21/2022,273,"$2,100,000.00",2536,$828.08,Lana Bell,Douglas Elliman,Brett Eaglstein,"Compass Florida, LLC",Trump Tower One 16001 Collins Ave 2704 Sunny I...,Trump Tower One,16001 Collins Ave 2704 Sunny Isles Beach,"(16001 Collins Ave APT 2704, Sunny Isles Beach...","(25.9244562, -80.1216846, 0.0)",25.924456,-80.121685,0.0
8,Trump Tower III\r15811 Collins Ave 1606\rSunny...,11/21/2022,424,"$1,900,000.00",2558,$742.77,Marco Cunha,Prestige Luxury Real Estate,Lana Bell,Douglas Elliman,Trump Tower III 15811 Collins Ave 1606 Sunny I...,Trump Tower III,15811 Collins Ave 1606 Sunny Isles Beach,"(15811 Collins Ave APT 1606, Sunny Isles Beach...","(25.9232597, -80.1218135, 0.0)",25.923260,-80.121814,0.0
9,Biscayne Beach\r2900 NE 7th Ave 3702\rMiami,11/21/2022,32,"$1,600,000.00",1698,$942.29,Bento Queiroz,"Compass Florida, LLC",Alison Zhuk,Douglas Elliman,Biscayne Beach 2900 NE 7th Ave 3702 Miami,Biscayne Beach,2900 NE 7th Ave 3702 Miami,"(2900 NE 7th Ave Unit 3702, Miami, FL 33137, U...","(25.8048485, -80.18598399999999, 0.0)",25.804848,-80.185984,0.0
10,0Key Colony\r201 Crandon Blvd 1028\rKey Biscayne,11/22/2022,129,"$1,450,000.00",1541,$940.95,Karen Holmquist,"Compass Florida, LLC",Kim Ellen Haug,Fortune Int'l Realty Key Biscayne Inc,0Key Colony 201 Crandon Blvd 1028 Key Biscayne,,0Key Colony 201 Crandon Blvd 1028 Key Biscayne,"(201 Crandon Blvd APT 1028, Key Biscayne, FL 3...","(25.

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[10,'building_name']=('Key Colony')

In [11]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('November 20th - November 26th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [14]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [16]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_111922
